In [1]:
import numpy as np
import os
import pyarrow
import sys
import json
import math
import mpl_utils
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import polars as pl
import xml.etree.ElementTree as ET

from xopen import xopen

In [2]:
sys.path.append("../functions")
import Demand_functions as dmd
import Supply_functions as sup

# Directories and Paths

In [3]:
# General directories
general_directory = '/Users/andre/Desktop/Cergy/'

berlin_directory = 'MATSim/matsim-berlin/input/v6.4/'

#pt_10pct_dir = "Python_Scripts/runs/dt_choice/"

In [4]:
# Get the passages information
EVENTS_PATH = (os.path.join(general_directory, berlin_directory, "parquet/passages.parquet"))

# Network info
NETWORK_PATH = (os.path.join(general_directory, berlin_directory, "berlin-v6.4-network.xml.gz"))

# Network

In [5]:
def read_nodes(network_path):

    node_data = []

    for event, elem in ET.iterparse(xopen(network_path, "r"), events=["start"]):
        if elem.tag == "node":
            try:
                node_id = (elem.attrib["id"])
                x = float(elem.attrib["x"])
                y = float(elem.attrib["y"])
                node_data.append((node_id, x, y))
            except Exception as e:
                print(f"Error al procesar nodo: {e}")
                
        if event == "end" and elem.tag == "node":
            elem.clear()
            
    nodes_df = pd.DataFrame(node_data, columns=["node_id", "x", "y"])
    
    # Limpiar solo los IDs que tienen "cluster"
    nodes_df.loc[nodes_df["node_id"].str.contains("cluster"), "node_id"] = (
        nodes_df.loc[nodes_df["node_id"].str.contains("cluster"), "node_id"]
        .str.replace("cluster_", "", regex=False)
        .str.split("_")
        .str[0]
    )
    nodes_df["node_id"] = nodes_df["node_id"].astype(int)
    
    return nodes_df

In [6]:
nodes_df = read_nodes(NETWORK_PATH)
links = sup.read_network(NETWORK_PATH)
edges = (sup.make_edges_df(links)).to_pandas()

More than two parallel edges
Maximum of 3 parallel edges
Handling 10 node pairs with more than 2 parallel edges...
Found 1832 parallel edges


In [7]:
# Source
nodes_source = nodes_df.rename(columns={"id": "source", "x": "x_source", "y": "y_source"})

# Target
nodes_target = nodes_df.rename(columns={"id": "target", "x": "x_target", "y": "y_target"})

In [11]:
# Add source and target coordinates
edges = edges.merge(nodes_source, left_on="source", right_on="node_id", how="left")
edges = edges.merge(nodes_target, left_on="target", right_on="node_id", how="left")

In [12]:
edges

,edge_id,MATSim_id,source,target,speed,length,lanes,speed_density.type,speed_density.capacity,speed_density.min_density,...,speed_density.beta,bottleneck_flow,constant_travel_time,overtaking,node_id_x,x_source,y_source,node_id_y,x_target,y_target
0,1,-1000055665,6773970060,267855868,4.634,28.01,1.0,FreeFlow,None,None,...,None,0.176111,0.955546,True,6.773970e+09,796573.10,5797464.90,2.678559e+08,796580.93,5797439.93
1,2,-1000098209,29266788,29266795,2.554,58.56,1.0,FreeFlow,None,None,...,None,0.222222,0.071261,True,2.926679e+07,800578.77,5830375.13,2.926680e+07,800629.32,5830399.93
2,3,-100012092,2288369966,1017979482,9.344,223.51,1.0,FreeFlow,None,None,...,None,0.526667,0.079837,True,2.288370e+09,807040.00,5714450.85,1.017979e+09,806856.97,5714552.62
3,4,-1000121005,9231911647,3869622346,10.460,136.97,1.0,FreeFlow,None,None,...,None,0.536944,0.905354,True,9.231912e+09,799342.30,5811143.42,3.869622e+09,799386.89,5811268.78
4,5,-1000268036,94673071,7829689846,7.133,69.88,1.0,FreeFlow,None,None,...,None,0.222222,0.203281,True,9.467307e+07,802003.09,5823888.17,7.829690e+09,801962.85,5823831.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204675,204676,28168333#2,10298089427,309400620,7.158,0.00,1.0,FreeFlow,None,None,...,None,0.222222,0.000000,True,NaN,NaN,NaN,3.094006e+08,789795.90,5811466.87
204676,204677,494438238#1,10298089428,30740827,10.712,0.00,1.0,FreeFlow,None,None,...,None,0.469444,0.000000,True,NaN,NaN,NaN,3.074083e+07,784346.04,5847127.88
204677,204678,8772077,10298089429,63044392,5.650,0.00,1.0,FreeFlow,None,None,...,None,0.176111,0.000000,True,NaN,NaN,NaN,6.304439e+07,787596.02,5852082.65
204678,204679,1060407617#1,10298089430,9480918655,7.691,0.00,1.0,FreeFlow,None,None,...,None,0.222222,0.000000,True,NaN,NaN,NaN,9.480919e+09,810340.21,5816066.54
